In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
from tensorflow import keras
from tensorflow.keras import *
import tensorflow_datasets as tfds
import math
import numpy as np
import numpy.random as rand

import tqdm
import math
import yaml
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sklearn
from sklearn.model_selection import train_test_split
from scipy import linalg as LA
import requests, json

In [2]:
# columns
"""
- historical crashes in the area (eg risk of other drivers)
- general visibility
- average space between cars
- average speed/severity of crashes in the area
- road weather (snow/hail/rain)
- road bumpiness/conditions
- traffic


- driving speed correctness: (driving speed) / (speed limit), or (speed limit) / (driving speed), whicheevr is greater
- hash acceleration
- harsh breaking
- harsh turning

returns:
future risk (crashes expected in the next 3 days)
"""
import math
import numpy as np
import numpy.random as rand
def create_person():
    """
    randomly generate location data and personal telemetrics data. 
    Use the data as described below to generate theoretical risk and future crashes. 
    
    Definitions:
        - L: overall location risk score
        - P: overall personal risk score
        - C: conditions subscore of location score
        - S: surrounding driver risk subscore of location score

    Description: 
        The total risk, aka the likelinhood of a claim being filed, is 
        calculated as P * log(L). 

            L is divided into C and S, which measure driving conditions and the risk 
            of surrounding drivers, respectively. We calculate it as Slog(C) + Clog(S)
                
                C consists of weather, visibility, and road bumpiness. 
                To capture the mild synergy relationship, we model this as 
                a*log(b)*log(c) + b*log(a)*log(c) + c*log(a)*log(b)

                S consists of number of crashes historically, space between cars,
                speed/severity of car crashes, and presence of traffic. 
                The score is calculated as:
                crash_num*crash_severity + traffic + space_bw_cars

            P, the personal risk score, is defined as (acceleration + braking + turning) ^ speed_ratio
        
        

    """
    data = {
    }

    data['historical_crashes'] = rand.geometric(p = .5) #average of .5 with some ppl in many crashes
    data['visbility'] = rand.uniform(0, 10) #usually at 10 miles with fog decreasing it rarely FIXME
    data['space_bt_cars'] = rand.uniform(5, 50) #pretty evenly distributed between 5 and 50
    data['crash_severity'] = rand.binomial(10, .5) #binomial distribution centered around 5
    data['rain_severity'] = rand.uniform(0, 10)
    data['snow_severity'] = rand.uniform(0, 10)
    data['road_bumpiness'] = rand.uniform(0, 10)
    data['traffic'] = rand.uniform(0, 10)

    data['speed_ratio'] = rand.uniform(.8, 1.5)
    data['harshness_acceleration'] = rand.uniform(0, 10)
    data['harshness_braking'] = rand.uniform(0, 10)
    data['harshness_turning'] = rand.uniform(0, 10)

    r = data['rain_severity']
    s = data['snow_severity']
    w = (r + s) / 2 + 1
    v = abs(data['visbility']) + 1
    b = abs(data['road_bumpiness']) + 1
    C = w * np.log(v) * np.log(b) + v * np.log(b) * np.log(w) + b * np.log(w) * np.log(v) + 1

    num = data['historical_crashes'] * 5
    sev = data['crash_severity']
    traff = data['traffic']
    space = data['space_bt_cars']/5

    S = num*sev + traff + space

    acc = data['harshness_acceleration']
    braking = data['harshness_braking']
    turn = data['harshness_turning']
    speed = data['speed_ratio']
    P = (acc + braking + turn) ** speed


    data['risk'] = np.log(S * np.log(C) + C * np.log(S)) + P
    
    if np.isnan(C) or np.isnan(S) or np.isnan(P):
      print("AAAAAAA")
    return data


In [3]:
train = pd.DataFrame([create_person() for i in range(10000)])
train_x = train.copy()
train_x.drop('risk', axis=1)
train_y = train['risk']

test = pd.DataFrame([create_person() for i in range(1000)])
test_x = test.copy()
test_x.drop('risk', axis=1)
display(train_x)
test_y = test[['risk']]


train_x = train_x.values
train_y = train_y.values
test_x = test_x.values
test_y = test_y.values

,historical_crashes,visbility,space_bt_cars,crash_severity,rain_severity,snow_severity,road_bumpiness,traffic,speed_ratio,harshness_acceleration,harshness_braking,harshness_turning,risk
0,1,7.571850,11.045002,3,2.387297,6.567724,6.837445,0.692386,0.946475,4.677314,1.998830,9.654693,19.835438
1,3,2.976315,19.754486,8,1.933007,2.710661,4.828740,4.721221,0.972034,7.923188,6.072179,5.922451,24.641304
2,1,1.329548,5.840638,4,7.712955,2.506022,0.618838,4.660236,1.345220,0.570147,4.262406,0.266937,13.327408
3,7,6.268501,16.534734,8,9.043879,7.944691,5.894924,8.933464,1.262691,8.092206,1.989580,9.101801,49.237044
4,3,2.691772,25.900309,4,3.413340,6.932319,8.949852,9.875657,0.947796,0.300162,9.609856,4.519141,18.877874
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2,4.063770,38.862886,2,5.535825,5.915008,4.533193,2.011938,1.325155,3.026712,2.087363,2.016227,19.206431
9996,2,6.892535,44.595933,6,6.474380,6.122652,2.635948,6.232664,1.420553,2.960490,5.691557,6.047710,51.818530
9997,1,4.553154,17.765506,4,5.100705,2.095946,5.408765,7.941794,0.943336,7.536634,7.707278,0.587191,19.200961
9998,1,6.499317,26.543680,7,5.741601,3.506436,5.094699,5.225593,0.843535,2.209452,3.993588,3.227676,12.732477


In [4]:
train_x.shape, train_y.shape

((10000, 13), (10000,))

In [5]:
model = Sequential([
    Input(shape=(13)),
    layers.Dense(128, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(1)          
])
opt = optimizers.Adam()
loss_func = losses.MeanSquaredError()
metric = ['mean_squared_error', 'mean_absolute_error'] 
model.compile(optimizer=opt,
              loss=loss_func,
              metrics=metric)

num_epochs = 30
model.fit(train_x, train_y, epochs=num_epochs)

Epoch 1/30
313/313 [==============================] - 1s 2ms/step - loss: 21.0137 - mean_squared_error: 21.0137 - mean_absolute_error: 1.4704
Epoch 2/30
313/313 [==============================] - 1s 2ms/step - loss: 0.0471 - mean_squared_error: 0.0471 - mean_absolute_error: 0.1672
Epoch 3/30
313/313 [==============================] - 1s 2ms/step - loss: 0.0285 - mean_squared_error: 0.0285 - mean_absolute_error: 0.1294
Epoch 4/30
313/313 [==============================] - 1s 2ms/step - loss: 0.0216 - mean_squared_error: 0.0216 - mean_absolute_error: 0.1135
Epoch 5/30
313/313 [==============================] - 1s 2ms/step - loss: 0.0176 - mean_squared_error: 0.0176 - mean_absolute_error: 0.1025
Epoch 6/30
313/313 [==============================] - 1s 2ms/step - loss: 0.0153 - mean_squared_error: 0.0153 - mean_absolute_error: 0.0956
Epoch 7/30
313/313 [==============================] - 1s 2ms/step - loss: 0.0137 - mean_squared_error: 0.0137 - mean_absolute_error: 0.0914
Epoch 8/30
313/313

In [6]:
model.evaluate(test_x, test_y)

32/32 [==============================] - 0s 1ms/step - loss: 0.1048 - mean_squared_error: 0.1048 - mean_absolute_error: 0.2816


[0.10475645959377289, 0.10475645959377289, 0.2815592885017395]